In [1]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [2]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('./log/default/best-85.pth'))
model.cuda()
def EuclideanDistances(A, B):
    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(), ED.shape[0] * ED.shape[1])


def cal_distance_image_real(images, labels):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)


def cal_distance_image_fake(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    labels=[]
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = output.data.max(1)[1]
        labels += [i for i in pred.cpu().numpy()]
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]

    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

import os
if not os.path.isdir('saved_models_{}'.format('bigan')):
    os.mkdir('saved_models_{}'.format('bigan'))
f = open('saved_models_{}/log_collapse1.txt'.format('bigan'), mode='w')
import torch.utils.data as Data
import cv2


from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

class BIGAN():
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # Build the encoder
        self.encoder = self.build_encoder()

        # The part of the bigan that trains the discriminator and encoder
        self.discriminator.trainable = False

        # Generate image from sampled noise
        z = Input(shape=(self.latent_dim, ))
        img_ = self.generator(z)

        # Encode image
        img = Input(shape=self.img_shape)
        z_ = self.encoder(img)

        # Latent -> img is fake, and img -> latent is valid
        fake = self.discriminator([z, img_])
        valid = self.discriminator([z_, img])

        # Set up and compile the combined model
        # Trains generator to fool the discriminator
        self.bigan_generator = Model([z, img], [fake, valid])
        self.bigan_generator.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
            optimizer=optimizer)

    def build_encoder(self):
        model = Sequential()
        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(self.latent_dim))

        model.summary()
        img = Input(shape=self.img_shape)
        z = model(img)

        return Model(img, z)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()
        z = Input(shape=(self.latent_dim,))
        gen_img = model(z)

        return Model(z, gen_img)

    def build_discriminator(self):

        z = Input(shape=(self.latent_dim,))
        img = Input(shape=self.img_shape)
        d_in = concatenate([z, Flatten()(img)])

        model = Dense(32 * 32 * 3)(d_in)
        model = Reshape((32, 32, 3))(model)
        model = Conv2D(32, kernel_size=3, strides=2, padding="same")(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.25)(model)
        model = Conv2D(64, kernel_size=3, strides=2, padding="same")(model)
        model = ZeroPadding2D(padding=((0, 1), (0, 1)))(model)
        model = BatchNormalization(momentum=0.8)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.25)(model)
        model = Conv2D(128, kernel_size=3, strides=2, padding="same")(model)
        model = BatchNormalization(momentum=0.8)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.25)(model)
        model = Conv2D(256, kernel_size=3, strides=1, padding="same")(model)
        model = BatchNormalization(momentum=0.8)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.25)(model)
        model = Flatten()(model)
        validity = Dense(1, activation="sigmoid")(model)

        return Model([z, img], validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (X_test, y_test) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0
        steps = []
        values = []

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            # idx = np.random.randint(0, X_train.shape[0], batch_size)
            for index in range(nb_batches):
                global_step += 1
                # progress_bar.update(index)

                # get a batch of real images
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]
                z = np.random.normal(size=(batch_size, self.latent_dim))
                imgs_ = self.generator.predict(z)

                # Select a random batch of images and encode
                # idx = np.random.randint(0, X_train.shape[0], batch_size)
                # imgs = X_train[idx]
                z_ = self.encoder.predict(image_batch)

                # Train the discriminator (img -> z is valid, z -> img is fake)
                d_loss_real = self.discriminator.train_on_batch([z_, image_batch], valid)
                d_loss_fake = self.discriminator.train_on_batch([z, imgs_], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator (z -> img is valid and img -> z is is invalid)
                g_loss = self.bigan_generator.train_on_batch([z, image_batch], [valid, fake])

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, global_step, d_loss[0],
                                                                                   100 * d_loss[1], g_loss[0]))
                sample_num=5000

                if global_step % sample_interval == 0:
                    self.mode_drop(X_test,y_test,sample_num, global_step)


    def mode_drop(self, x_test,y_test,sample_num, global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, 100))
        # sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('step:' + str(global_step))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')

if __name__ == '__main__':
    bigan = BIGAN()
    bigan.train(epochs=50, batch_size=64, sample_interval=200)


Using TensorFlow backend.
Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8192)              827392    
_________________________________________________________________
reshape_2 (Reshape)          (None, 8, 8, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
batch_normalization_4 (Batch (None, 16, 16, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 128)       0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 32, 32, 128)       0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 1.071894, acc: 42.19%] [G loss: 8.116537]
epoch:0 step:2 [D loss: 0.284235, acc: 86.72%] [G loss: 9.533644]
epoch:0 step:3 [D loss: 0.083295, acc: 98.44%] [G loss: 9.846010]
epoch:0 step:4 [D loss: 0.214717, acc: 94.53%] [G loss: 9.532492]
epoch:0 step:5 [D loss: 0.139198, acc: 96.88%] [G loss: 9.261973]
epoch:0 step:6 [D loss: 0.617191, acc: 71.09%] [G loss: 7.559661]
epoch:0 step:7 [D loss: 0.602661, acc: 78.91%] [G loss: 5.743185]
epoch:0 step:8 [D loss: 2.052404, acc: 35.94%] [G loss: 3.717464]
epoch:0 step:9 [D loss: 0.232312, acc: 89.06%] [G loss: 3.810910]
epoch:0 step:10 [D loss: 1.423892, acc: 46.09%] [G loss: 6.249705]
epoch:0 step:11 [D loss: 0.547789, acc: 78.12%] [G loss: 6.263858]
epoch:0 step:12 [D loss: 0.477442, acc: 81.25%] [G loss: 5.751565]
epoch:0 step:13 [D loss: 0.163575, acc: 89.84%] [G loss: 5.365552]
epoch:0 step:14 [D loss: 0.521411, acc: 74.22%] [G loss: 8.290746]
epoch:0 step:15 [D loss: 0.401482, acc: 85.94%] [G loss: 9.667978]
epoc

epoch:0 step:124 [D loss: 0.028586, acc: 100.00%] [G loss: 5.222867]
epoch:0 step:125 [D loss: 0.052866, acc: 99.22%] [G loss: 5.745964]
epoch:0 step:126 [D loss: 0.083064, acc: 99.22%] [G loss: 6.625940]
epoch:0 step:127 [D loss: 0.069275, acc: 100.00%] [G loss: 5.842546]
epoch:0 step:128 [D loss: 0.053383, acc: 100.00%] [G loss: 4.819181]
epoch:0 step:129 [D loss: 0.035443, acc: 99.22%] [G loss: 5.266744]
epoch:0 step:130 [D loss: 0.035688, acc: 100.00%] [G loss: 4.152209]
epoch:0 step:131 [D loss: 0.388503, acc: 82.03%] [G loss: 7.510274]
epoch:0 step:132 [D loss: 0.047700, acc: 99.22%] [G loss: 7.422567]
epoch:0 step:133 [D loss: 0.156401, acc: 92.97%] [G loss: 6.356783]
epoch:0 step:134 [D loss: 0.022332, acc: 100.00%] [G loss: 5.880509]
epoch:0 step:135 [D loss: 0.089378, acc: 97.66%] [G loss: 5.943245]
epoch:0 step:136 [D loss: 0.037286, acc: 99.22%] [G loss: 6.202609]
epoch:0 step:137 [D loss: 0.561871, acc: 75.78%] [G loss: 9.337641]
epoch:0 step:138 [D loss: 0.254672, acc: 89

...

epoch:49 step:38551 [D loss: 0.423764, acc: 69.53%] [G loss: 8.530737]
epoch:49 step:38552 [D loss: 0.117837, acc: 98.44%] [G loss: 5.181237]
epoch:49 step:38553 [D loss: 0.477706, acc: 73.44%] [G loss: 3.767008]
epoch:49 step:38554 [D loss: 0.053201, acc: 100.00%] [G loss: 8.114046]
epoch:49 step:38555 [D loss: 0.181742, acc: 99.22%] [G loss: 4.745957]
epoch:49 step:38556 [D loss: 0.039271, acc: 100.00%] [G loss: 4.462261]
epoch:49 step:38557 [D loss: 0.279723, acc: 87.50%] [G loss: 6.286064]
epoch:49 step:38558 [D loss: 0.567253, acc: 60.94%] [G loss: 9.052530]
epoch:49 step:38559 [D loss: 0.226586, acc: 95.31%] [G loss: 9.158396]
epoch:49 step:38560 [D loss: 1.552159, acc: 50.78%] [G loss: 6.234169]
epoch:49 step:38561 [D loss: 0.051991, acc: 100.00%] [G loss: 8.151899]
epoch:49 step:38562 [D loss: 0.301250, acc: 83.59%] [G loss: 8.110260]
epoch:49 step:38563 [D loss: 0.253228, acc: 89.84%] [G loss: 6.672089]
epoch:49 step:38564 [D loss: 0.344194, acc: 84.38%] [G loss: 7.311913]
epo

epoch:49 step:38664 [D loss: 0.189158, acc: 97.66%] [G loss: 4.872511]
epoch:49 step:38665 [D loss: 0.026453, acc: 100.00%] [G loss: 5.773404]
epoch:49 step:38666 [D loss: 0.334460, acc: 94.53%] [G loss: 6.437474]
epoch:49 step:38667 [D loss: 0.538228, acc: 67.97%] [G loss: 8.666069]
epoch:49 step:38668 [D loss: 0.245148, acc: 96.09%] [G loss: 7.552814]
epoch:49 step:38669 [D loss: 0.006729, acc: 100.00%] [G loss: 6.136456]
epoch:49 step:38670 [D loss: 0.013069, acc: 100.00%] [G loss: 9.730010]
epoch:49 step:38671 [D loss: 1.074475, acc: 50.78%] [G loss: 5.950663]
epoch:49 step:38672 [D loss: 0.036213, acc: 100.00%] [G loss: 4.977769]
epoch:49 step:38673 [D loss: 1.449870, acc: 50.00%] [G loss: 5.730835]
epoch:49 step:38674 [D loss: 0.048322, acc: 100.00%] [G loss: 9.248395]
epoch:49 step:38675 [D loss: 0.079308, acc: 100.00%] [G loss: 11.329140]
epoch:49 step:38676 [D loss: 0.669197, acc: 57.03%] [G loss: 8.424559]
epoch:49 step:38677 [D loss: 0.148571, acc: 97.66%] [G loss: 3.063459]

epoch:49 step:38782 [D loss: 0.039981, acc: 100.00%] [G loss: 4.255907]
epoch:49 step:38783 [D loss: 0.118269, acc: 100.00%] [G loss: 5.453391]
epoch:49 step:38784 [D loss: 0.156674, acc: 99.22%] [G loss: 6.586004]
epoch:49 step:38785 [D loss: 0.226852, acc: 92.19%] [G loss: 4.651856]
epoch:49 step:38786 [D loss: 0.236631, acc: 89.06%] [G loss: 12.362565]
epoch:49 step:38787 [D loss: 1.241973, acc: 32.03%] [G loss: 8.816689]
epoch:49 step:38788 [D loss: 0.370883, acc: 82.03%] [G loss: 7.244182]
epoch:49 step:38789 [D loss: 1.178324, acc: 50.78%] [G loss: 5.758637]
epoch:49 step:38790 [D loss: 0.059728, acc: 100.00%] [G loss: 6.455717]
epoch:49 step:38791 [D loss: 0.131316, acc: 100.00%] [G loss: 6.418021]
epoch:49 step:38792 [D loss: 0.546852, acc: 61.72%] [G loss: 8.198114]
epoch:49 step:38793 [D loss: 0.255787, acc: 88.28%] [G loss: 8.474720]
epoch:49 step:38794 [D loss: 0.151264, acc: 99.22%] [G loss: 3.539345]
epoch:49 step:38795 [D loss: 0.085886, acc: 99.22%] [G loss: 3.142316]
e

epoch:49 step:38897 [D loss: 0.260278, acc: 97.66%] [G loss: 6.999849]
epoch:49 step:38898 [D loss: 0.028922, acc: 100.00%] [G loss: 7.855912]
epoch:49 step:38899 [D loss: 0.010246, acc: 100.00%] [G loss: 9.136208]
epoch:49 step:38900 [D loss: 0.134998, acc: 100.00%] [G loss: 9.387511]
epoch:49 step:38901 [D loss: 0.036227, acc: 100.00%] [G loss: 7.029159]
epoch:49 step:38902 [D loss: 0.074968, acc: 100.00%] [G loss: 5.934005]
epoch:49 step:38903 [D loss: 0.230944, acc: 94.53%] [G loss: 5.348897]
epoch:49 step:38904 [D loss: 0.022346, acc: 100.00%] [G loss: 5.916573]
epoch:49 step:38905 [D loss: 1.225581, acc: 32.03%] [G loss: 6.450384]
epoch:49 step:38906 [D loss: 0.348525, acc: 83.59%] [G loss: 4.882794]
epoch:49 step:38907 [D loss: 0.156505, acc: 96.88%] [G loss: 6.306778]
epoch:49 step:38908 [D loss: 0.441782, acc: 69.53%] [G loss: 5.093012]
epoch:49 step:38909 [D loss: 0.232287, acc: 95.31%] [G loss: 3.488748]
epoch:49 step:38910 [D loss: 0.046697, acc: 100.00%] [G loss: 8.076378]

epoch:49 step:39011 [D loss: 0.060452, acc: 100.00%] [G loss: 5.075298]
epoch:49 step:39012 [D loss: 0.038305, acc: 100.00%] [G loss: 3.982856]
epoch:49 step:39013 [D loss: 0.468784, acc: 68.75%] [G loss: 8.088291]
epoch:49 step:39014 [D loss: 0.015616, acc: 100.00%] [G loss: 3.510086]
epoch:49 step:39015 [D loss: 0.251260, acc: 96.88%] [G loss: 7.022685]
epoch:49 step:39016 [D loss: 0.301280, acc: 87.50%] [G loss: 6.738506]
epoch:49 step:39017 [D loss: 1.113481, acc: 41.41%] [G loss: 6.728136]
epoch:49 step:39018 [D loss: 0.131283, acc: 99.22%] [G loss: 5.010524]
epoch:49 step:39019 [D loss: 0.591739, acc: 60.16%] [G loss: 9.147638]
epoch:49 step:39020 [D loss: 0.243865, acc: 92.97%] [G loss: 4.156609]
epoch:49 step:39021 [D loss: 0.503930, acc: 75.78%] [G loss: 7.829105]
epoch:49 step:39022 [D loss: 0.177347, acc: 96.88%] [G loss: 5.180837]
epoch:49 step:39023 [D loss: 0.348344, acc: 84.38%] [G loss: 7.755022]
epoch:49 step:39024 [D loss: 0.336666, acc: 85.16%] [G loss: 9.306794]
epo